Trabajo Final Python

In [1]:
#Ejercicio práctico
#Recibirán 4 archivos, 3 csv (Data de nuestro sistema marsh) y 1 xlsx (Data de la aseguradora)
#El objetivo final es ver qué tiene la data de la aseguradora que esté en la data de marsh.

In [2]:
#Importamos las librerías
import pandas as pd
import glob
import os

-Cargar los 4 archivos a Python (los 3 csv cargarlos con bucle for)

In [3]:
# 

# Ruta de la carpeta que contiene los archivos CSV
carpeta = 'C:/OTMX/Documentation/Files_Python_Santiago/Sesión_Final/Inputs'

# Obtener la lista de archivos en la carpeta
archivos = os.listdir(carpeta)
print(archivos) #imprimir esa lista

['ASSA-MAYO.xlsx', 'ProduccionOnlineAssa202403.csv', 'ProduccionOnlineAssa202404.csv', 'ProduccionOnlineAssa202405.csv']


In [4]:
# Crear un DataFrame vacío para almacenar los datos
#datos = pd.DataFrame()

dataframes_csv = []
dataframes_xlsx = []


# Iterar sobre cada archivo en la carpeta
for archivo in archivos:
    # Verificar si el archivo es un archivo CSV
    if archivo.endswith('.csv'):
        # Cargar el archivo CSV en un DataFrame
        ruta_archivo = os.path.join(carpeta, archivo)
        df_csv = pd.read_csv(ruta_archivo, delimiter='|')

        # Establecer la primera fila como encabezado
        #datos.columns = datos.iloc[0]
        #datos = datos[1:]
        # Agregar los datos al DataFrame principal
        #datos = datos.append(df, ignore_index=True)
        dataframes_csv.append(df_csv)

            # Verificar si el archivo es un archivo XLSX
    elif archivo.endswith('.xlsx'):
        # Cargar el archivo XLSX en un DataFrame
        ruta_archivo = os.path.join(carpeta, archivo)
        df_xlsx = pd.read_excel(ruta_archivo)
        
        # Agregar los datos al DataFrame de archivos XLSX
        dataframes_xlsx.append(df_xlsx)


marsh = pd.concat(dataframes_csv)
asg = pd.concat(dataframes_xlsx)



-De la información de marsh dejar solamente estas columnas : 'NumeroPoliza','Estatus', 'FechaEmision', 'FechaInicioVigencia', 'FechaFinVigencia', 'Prima', 'TotalPrima'

In [5]:
marsh.columns


Index(['fecha_archivo', '﻿IdProduccion', 'IdPoliza', 'Apellido', 'Nombre',
       'TipoIdentificacion', 'Cedula', 'Email', 'FechaNacimiento', 'Telefono1',
       'Telefono2', 'Celular', 'Sexo', 'Estatus', 'FechaEmision',
       'FechaInicioVigencia', 'FechaFinVigencia', 'Prima', 'TotalPrima',
       'Producto', 'NumeroPoliza', 'FormaPago', 'FrecuenciaPago',
       'CantidadLetras', 'LinkDoc', 'TipoDeEndoso', 'CodEndoso',
       'PorcComision'],
      dtype='object')

In [6]:

marsh = marsh[['NumeroPoliza','Estatus', 'FechaEmision', 'FechaInicioVigencia', 'FechaFinVigencia', 'Prima', 'TotalPrima']]

#-Convertir todos los nombres de columnas a mayúsculas

In [7]:
# Convertir los nombres de las columnas a mayúsculas
marsh.columns = marsh.columns.str.upper()

#-Hacer el left Join (aseguradora <- marsh)
#Recordar que se pueden cambiar los nombres de las columnas para tenerlos iguales y hacer el cruce más sencillo.

NÚMERO DE PÓLIZA y TIPO POLIZA= En la data de la aseguradora aparece como POLIZA y en la data de marsh aparece como NUMEROPOLIZA.


In [8]:
#Renombrar columna para el cruce
marsh = marsh.rename(columns={'NUMEROPOLIZA':'POLIZA'})



TIPO= En la data de la aseguradora aparece como TIPO y en la data de marsh como ESTATUS.

In [9]:
#Renombrar columna para el cruce
marsh = marsh.rename(columns={'ESTATUS':'TIPO'})

In [10]:

#Es importante tener en cuenta que el tipo de movimiento si es nueva o renovación aparece diferente en los dos dataframes, homologarlos primero para poder
#hacer el cruce, en la data de la aseguradora aparece como EMI y EMR, reemplazar EMI por NUEVA y EMR por RENOVACION para poder hacer el cruce.

# Reemplazar los valores en la columna 'TIPO'
asg['TIPO'] = asg['TIPO'].replace({'EMI': 'NUEVA', 'EMR': 'RENOVACION'})

In [11]:

#También verificar que las columnas de POLIZA no tengan algún caracter diferente que no permita cruzarlas.
asg.dtypes

TIPO                 object
POLIZA               object
DESDE                object
HASTA                object
Canal de Dist.       object
FACTURACION          object
VIGENCIA EFECTIVA    object
PRIMA                object
dtype: object

In [12]:
marsh.dtypes

POLIZA                  object
TIPO                    object
FECHAEMISION            object
FECHAINICIOVIGENCIA     object
FECHAFINVIGENCIA        object
PRIMA                  float64
TOTALPRIMA             float64
dtype: object

In [13]:
#Para la aseguradora

#Convertir los tipos de fecha
asg['DESDE'] = pd.to_datetime(asg['DESDE']).dt.strftime('%d-%m-%Y')
# Ahora que la columna es de tipo fecha, puedes darle formato utilizando la función strftime
#asg['DESDE'] = asg['DESDE'].dt.strftime('%d-%m-%Y')

C:\Users\u1165269\AppData\Local\Temp\ipykernel_26264\3762069699.py:4: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  asg['DESDE'] = pd.to_datetime(asg['DESDE']).dt.strftime('%d-%m-%Y')


In [14]:
#Para marsh
#Convertir los tipos de fecha
marsh['FECHAINICIOVIGENCIA'] = pd.to_datetime(marsh['FECHAINICIOVIGENCIA'])
# Ahora que la columna es de tipo fecha, puedes darle formato utilizando la función strftime
marsh['FECHAINICIOVIGENCIA'] = marsh['FECHAINICIOVIGENCIA'].dt.strftime('%d-%m-%Y')


In [15]:
# Quitamos el guión medio
asg['POLIZA'] = asg['POLIZA'].str.replace('-', '')
#Se hacen los cruces
df_All = pd.merge(asg,marsh,on='POLIZA',how='left',suffixes=['_asg','_marsh'])

#-Crear una columna nueva que me compare si la columna "DESDE" es igual o diferente a la columna "FECHAINICIOVIGENCIA" (verificar los tipos de datos para poder hacer la comparación)

In [16]:
import numpy as np

condiciones = [df_All['DESDE'] == df_All['FECHAINICIOVIGENCIA']]
choices = ['1']

df_All['VAL'] = np.select(condiciones,choices,default='0')

In [18]:
#-Exportar en un excel con varias hojas, la primera el cruce, la segunda la data completa de la aseguradora y la tercera la data completa de marsh

import xlsxwriter

writer = pd.ExcelWriter('C:/OTMX/Outputs/df_All.xlsx')

df_All.to_excel(writer,sheet_name='All',index=False)
asg.to_excel(writer,sheet_name='ASG',index=False)
marsh.to_excel(writer,sheet_name='MARSH',index=False)

writer.close()


